In [15]:
import os
import pandas as pd
import re

from kedro.pipeline import *
from kedro.io import *
from kedro.runner import *

import pickle
import os

import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import product



In [16]:
csv_file_path = r'C:\Users\jrubio\vivalto_data\data\04_feature\df_number_of_id_vivalto_translate.csv'
df = pd.read_csv(csv_file_path)
df

,WEEK_LABEL,DIAGNOSIS_CATEGORY,AGE_CLASS,RSS_DURATION_CLASS,N_CASES
0,2023-07-17,COVID-19,65 years and older,5 days and more,3
1,2023-07-17,COVID-19,65 years and older,Less than 2 days,0
2,2023-07-17,FLU,65 years and older,5 days and more,0
3,2023-07-17,GENERAL_RI,65 years and older,5 days and more,44
4,2023-07-17,GENERAL_RI,65 years and older,Less than 2 days,2
...,...,...,...,...,...
215,2023-10-09,COVID-19,65 years and older,Less than 2 days,0
216,2023-10-09,GENERAL_RI,65 years and older,Less than 2 days,0
217,2023-10-09,GENERAL_RI,65 years and older,[2 - 5[ days,0
218,2023-10-09,GENERAL_RI,[50 - 64[ year(s),Less than 2 days,0


In [17]:
result_df = df.sort_values(by=['DIAGNOSIS_CATEGORY', 'AGE_CLASS', 'RSS_DURATION_CLASS', 'WEEK_LABEL'], ascending=[True, True, True, True])
result_df

,WEEK_LABEL,DIAGNOSIS_CATEGORY,AGE_CLASS,RSS_DURATION_CLASS,N_CASES
0,2023-07-17,COVID-19,65 years and older,5 days and more,3
13,2023-07-24,COVID-19,65 years and older,5 days and more,0
26,2023-07-31,COVID-19,65 years and older,5 days and more,0
49,2023-08-07,COVID-19,65 years and older,5 days and more,2
61,2023-08-14,COVID-19,65 years and older,5 days and more,4
...,...,...,...,...,...
175,2023-09-18,RI_OTHER_VIRUS,[50 - 64[ year(s),5 days and more,0
196,2023-09-25,RI_OTHER_VIRUS,[50 - 64[ year(s),5 days and more,0
48,2023-07-31,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0
80,2023-08-14,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0


In [18]:
# Définissez les conditions et les valeurs correspondantes
result_df['COMBI'] = result_df['DIAGNOSIS_CATEGORY'] + ' | ' + result_df['AGE_CLASS']+ ' | ' + result_df['RSS_DURATION_CLASS']

# Créer un graphique en bar plot pour le nombre de valeurs par année par combinaison DIAGNOSIS_CATEGORY et CLASS_AGE
result_df['WEEK_LABEL'] = pd.to_datetime(result_df['WEEK_LABEL'])
result_df['YEAR'] = result_df['WEEK_LABEL'].dt.year

result_df = result_df.sort_values(by='COMBI')
result_df = result_df.drop(columns='COMBI')
result_df = result_df.drop(columns='YEAR')
    
weeks = pd.date_range(start=result_df['WEEK_LABEL'].min(), end=result_df['WEEK_LABEL'].max(), freq='W-Mon')


categories_age = result_df['AGE_CLASS'].unique()
diagnosis_categories = result_df['DIAGNOSIS_CATEGORY'].unique()
rss_duration_categories = result_df['RSS_DURATION_CLASS'].unique()

combinations = list(product(weeks, diagnosis_categories, categories_age, rss_duration_categories))

columns = ['WEEK_LABEL', 'DIAGNOSIS_CATEGORY', 'AGE_CLASS','RSS_DURATION_CLASS']
all_combinations_df = pd.DataFrame(combinations, columns=columns)

merged_df = pd.merge(all_combinations_df, result_df, on=['WEEK_LABEL', 'AGE_CLASS','RSS_DURATION_CLASS','DIAGNOSIS_CATEGORY'], how='outer')

merged_df = merged_df.sort_values(by='WEEK_LABEL')
merged_df['N_CASES'] = merged_df['N_CASES'].fillna(0)

merged_df = merged_df.sort_values(by=['DIAGNOSIS_CATEGORY', 'AGE_CLASS', 'RSS_DURATION_CLASS', 'WEEK_LABEL'], ascending=[True, True, True, True])
merged_df

,WEEK_LABEL,DIAGNOSIS_CATEGORY,AGE_CLASS,RSS_DURATION_CLASS,N_CASES
0,2023-07-17,COVID-19,65 years and older,5 days and more,3.0
60,2023-07-24,COVID-19,65 years and older,5 days and more,0.0
120,2023-07-31,COVID-19,65 years and older,5 days and more,0.0
180,2023-08-07,COVID-19,65 years and older,5 days and more,2.0
240,2023-08-14,COVID-19,65 years and older,5 days and more,4.0
...,...,...,...,...,...
536,2023-09-11,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0.0
596,2023-09-18,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0.0
656,2023-09-25,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0.0
716,2023-10-02,RI_OTHER_VIRUS,[50 - 64[ year(s),[2 - 5[ days,0.0


In [19]:
df = pd.DataFrame(merged_df)

# Spécifiez l'emplacement du fichier CSV
chemin_fichier = "C:\\Users\\jrubio\\vivalto_data\\data\\04_feature\\merged_df.csv"

# Enregistrez le DataFrame dans un fichier CSV
df.to_csv(chemin_fichier)

In [22]:
diagnosis_counts = df['DIAGNOSIS_CATEGORY'].value_counts()
pd.DataFrame(diagnosis_counts)

,count
DIAGNOSIS_CATEGORY,
COVID-19,195
FLU,195
GENERAL_RI,195
RI_OTHER_VIRUS,195
